- PassengerId 整型变量，标识乘客的ID，递增变量，对预测无帮助
- Survived 整型变量，标识该乘客是否幸存。0表示遇难，1表示幸存。将其转换为factor变量比较方便处理
- Pclass 整型变量，标识乘客的社会-经济状态，1代表Upper，2代表Middle，3代表Lower
- Name 字符型变量，除包含姓和名以外，还包含Mr.
- Mrs. Dr.这样的具有西方文化特点的信息
- Sex 字符型变量，标识乘客性别，适合转换为factor类型变量
- Age 整型变量，标识乘客年龄，有缺失值
- SibSp 整型变量，代表兄弟姐妹及配偶的个数。其中Sib代表Sibling也即兄弟姐妹，Sp代表Spouse也即配偶
- Parch 整型变量，代表父母或子女的个数。其中Par代表Parent也即父母，Ch代表Child也即子女
- Ticket 字符型变量，代表乘客的船票号 Fare 数值型，代表乘客的船票价
- Cabin 字符型，代表乘客所在的舱位，有缺失值
- Embarked 字符型，代表乘客登船口岸，适合转换为factor型变量

In [94]:
import numpy as np
from collections import Counter,defaultdict
import pandas #ipython notebook
import pandas as pd

train = pandas.read_csv("train.csv")
train.head(5)
test=pandas.read_csv("test.csv")



In [95]:
#处理 Embarked 特征
Embarked_list= train['Embarked'].unique()
train['Embarked'].fillna('S',inplace=True)
test['Embarked'].fillna('S',inplace=True)
train['Embarked']=train['Embarked'].astype('category')
test['Embarked']=test['Embarked'].astype('category')


In [96]:
# 分割姓名称谓并统计数据
import re #正则表达式
name_pattern=re.compile('[A-Za-z]+\.?')
name_list= train['Name'].to_numpy()
words=Counter(word for name in name_list for word in re.findall(name_pattern,name))
words=list((-c,key) for key,c in words.items())
words.sort()
#展示name中单词的统计频次
for c,word in words[:50]:
    print("{:d}\t{}".format(-c,word))


517	Mr.
182	Miss.
125	Mrs.
64	William
44	John
40	Master.
35	Henry
24	Charles
24	George
24	James
22	Thomas
20	Mary
18	Edward
17	Anna
16	Joseph
15	Elizabeth
15	Frederick
15	Johan
14	Richard
13	Arthur
13	Samuel
12	Alfred
12	Margaret
11	Alexander
11	Maria
11	Peter
10	Jr
10	Robert
9	Andersson
9	Ernest
9	Karl
9	Leonard
9	O
8	Albert
8	Alice
8	Annie
8	H
8	J
8	Martin
8	Victor
7	Catherine
7	David
7	Dr.
7	Frank
7	Helen
7	Kate
7	Sage
7	de
6	Carter
6	E


In [97]:
#可以提取称谓作为新特征
title_list=tuple(w for c,w in filter(lambda cw:'.'==cw[1][-1] and -cw[0]>6,words))
def get_title(name_list):
    titleMatrix=[None]*len(title_list)
    for i,key in enumerate(title_list):
        titleMatrix[i]=list(int(key in name) for name in name_list)
    titleMatrix=pd.concat(tuple(map(pd.Series,titleMatrix)),axis=1)
    titleMatrix.columns=title_list
    return titleMatrix
train_titles=get_title(name_list)
test_titles=get_title(test['Name'].to_numpy())

In [98]:
# 去掉无效特征并添加新特征
train.drop(['Name','Ticket','Cabin'],axis=1)
train=pd.concat([train,train_titles],axis=1)
test.drop(['Name','Ticket','Cabin'],axis=1)
test=pd.concat([train,test_titles],axis=1)

In [106]:
# train.describe()
# 训练年龄预测模型
naAge_train=train['Age'].isna()
naAge_test=test['Age'].isna()
# 注意预测age不能使用Survived属性
def getXYofAge(data:pd.DataFrame):
    column=list(data.columns)
    print(column)
    column.remove('Age')
    return data[column],data['Age']
    # return np.ndarray(data[column],dtype='float64'),np.ndarray(data['Age'],dtype='float64')
AgeTrainX,AgeTrainY=getXYofAge(train[test.columns].loc[~naAge_train])
AgeTestX,AgeTestY=getXYofAge(test.loc[~naAge_test])

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Mr.', 'Miss.', 'Mrs.', 'Master.', 'Dr.', 'Mr.', 'Miss.', 'Mrs.', 'Master.', 'Dr.']
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Mr.', 'Miss.', 'Mrs.', 'Master.', 'Dr.', 'Mr.', 'Miss.', 'Mrs.', 'Master.', 'Dr.']


In [100]:
# 对年龄缺失值进行预测(包括原训练组和测试组)
params={
    'n_estimators':200,
    'objective':'binary:logistic',
    'max_depth':4,
    'learning_rate':0.1
}
# python 语法，字典对象前加两个*可以转化成以key为参数名且对应value为参数值带入到方法参数列表中。
import xgboost as xgb
def Age_prediction(x_train,y_train):
    model=xgb.XGBClassifier(**params).fit(x_train,y_train)

NameError: name 'x_train' is not defined